In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from os import getcwd, chdir
import pandas as pd
import pickle as pk
fpath = getcwd()
print(fpath)
chdir(fpath)

/content


In [ ]:
posp = "/content/drive/MyDrive/colab/nus/5004 PLP/nmsm/datasets/Data/rt-polarity-utf8.pos"
negp = "/content/drive/MyDrive/colab/nus/5004 PLP/nmsm/datasets/Data/rt-polarity-utf8.neg"
poslines = open(posp, 'r').read().splitlines()
neglines = open(negp, 'r').read().splitlines()

print("No of positive reviews " + str(len(poslines)))
print("No of negative reviews " + str(len(neglines)))

No of positive reviews 5331
No of negative reviews 5331


In [ ]:
N=4800
poslinesTrain = poslines[:N]
neglinesTrain = neglines[:N]
poslinesTest = poslines[N:]
neglinesTest = neglines[N:]

In [ ]:
trainset = [(x, 1) for x in poslinesTrain] + [(x, -1) for x in neglinesTrain]
testset = [(x, 1) for x in poslinesTest] + [(x, -1) for x in neglinesTest]


In [ ]:
poswords = {}
negwords = {}

for line, label in trainset:
    for word in line.split():
        if label == 1:
            poswords[word] = poswords.get(word, 0) + 1
        else:
            negwords[word] = negwords.get(word, 0) + 1


In [ ]:
print ("No of negative words in trainset " + str(len(negwords)))
print ("No of positive words in trainset " + str(len(poswords)))

No of negative words in trainset 13663
No of positive words in trainset 13305


In [ ]:
posFreqs = {k: poswords[k] for k in list(poswords)[:5]}
print(posFreqs)
negFreqs = {k: negwords[k] for k in list(negwords)[:5]}
print(negFreqs)

{'the': 4554, 'rock': 11, 'is': 1541, 'destined': 5, 'to': 1766}
{'simplistic': 15, ',': 4256, 'silly': 52, 'and': 2371, 'tedious': 26}


In [ ]:
wrong = 0
pred_list = []
actual = []
for line, label in testset:
    totpos, totneg = 0.0, 0.0
    for word in line.split():
        a = poswords.get(word, 0.0) + 1.0
        b = negwords.get(word, 0.0) + 1.0

        totpos += a/(a+b)
        totneg += b/(a+b)
    
    prediction = 1
    if totneg > totpos:
        prediction = -1
    pred_list.append(prediction)
    actual.append(label)
    if prediction != label:
        wrong += 1
        print ('ERROR: %s posscore = %.2f negscore=%.2f' % (line, totpos, totneg))
    else:
        pass


ERROR: it's like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .  posscore = 11.84 negscore=12.16
ERROR: weighty and ponderous but every bit as filling as the treat of the title .  posscore = 7.34 negscore=7.66
ERROR: a real audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .  posscore = 13.81 negscore=14.19
ERROR: exposing the ways we fool ourselves is one hour photo's real strength .  posscore = 6.29 negscore=6.71
ERROR: at its worst , the movie is pretty diverting ; the pity is that it rarely achieves its best .  posscore = 9.92 negscore=10.08
ERROR: as it turns out , you can go home again .  posscore = 5.31 negscore=5.69
ERROR: devotees of star trek ii : the wrath of khan will feel a nagging sense of deja vu , and the grandeur of the best next generation episodes is lacking .  posscore = 14.84 negscore=16

In [ ]:
print ('error rate is %f' % (1.0*wrong/len(testset),))
print ('No of wrongs ' + str(wrong))
print ('Size of test set ' + str(len(testset)))

error rate is 0.209981
No of wrongs 223
Size of test set 1062


In [ ]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('punkt')

from nltk.corpus import sentiwordnet as swn
from scipy import mean
from nltk.tokenize import word_tokenize as wt

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
synsets = swn.senti_synsets('fast')

for syn in synsets:
    print(str(syn))

<fast.n.01: PosScore=0.0 NegScore=0.0>
<fast.v.01: PosScore=0.0 NegScore=0.0>
<fast.v.02: PosScore=0.0 NegScore=0.0>
<fast.a.01: PosScore=0.0 NegScore=0.0>
<fast.a.02: PosScore=0.0 NegScore=0.0>
<fast.a.03: PosScore=0.0 NegScore=0.0>
<fast.s.04: PosScore=0.0 NegScore=0.0>
<fast.s.05: PosScore=0.125 NegScore=0.0>
<debauched.s.01: PosScore=0.375 NegScore=0.625>
<flying.s.02: PosScore=0.25 NegScore=0.25>
<fast.s.08: PosScore=0.0 NegScore=0.0>
<firm.s.10: PosScore=0.5 NegScore=0.0>
<fast.s.10: PosScore=0.25 NegScore=0.0>
<fast.r.01: PosScore=0.0 NegScore=0.0>
<fast.r.02: PosScore=0.0 NegScore=0.0>


In [ ]:
def get_pos_neg_score(word, metric):
    posi = [0.0]
    negi = [0.0]
    synsets = swn.senti_synsets(word)
    for syn in synsets:
        posi.append(syn.pos_score())
        negi.append(syn.neg_score())
    if metric =="Mean":
        pos = mean(posi)
        neg = mean(negi)
    else:
        pos = max(posi)
        neg = max(negi)
    return pos, neg

In [ ]:
get_pos_neg_score("fast", "Mean")

<ipython-input-19-5d466d5344c0>:9: DeprecationWarning: scipy.mean is deprecated and will be removed in SciPy 2.0.0, use numpy.mean instead
  pos = mean(posi)
<ipython-input-19-5d466d5344c0>:10: DeprecationWarning: scipy.mean is deprecated and will be removed in SciPy 2.0.0, use numpy.mean instead
  neg = mean(negi)


(0.09375, 0.0546875)

In [ ]:
pred = []
actual = []
for line, label in testset:
    pos_rev = neg_rev = 0
    for word in wt(line):
        pos, neg = get_pos_neg_score(word, "Mean")
        pos_rev += pos
        neg_rev += neg
    if pos_rev > neg_rev:
        lab = 1
    else:
        lab = -1
    pred.append(lab)
    actual.append(label)

actuals = pd.Series(actual)
predicted = pd.Series(pred)

<ipython-input-19-5d466d5344c0>:9: DeprecationWarning: scipy.mean is deprecated and will be removed in SciPy 2.0.0, use numpy.mean instead
  pos = mean(posi)
<ipython-input-19-5d466d5344c0>:10: DeprecationWarning: scipy.mean is deprecated and will be removed in SciPy 2.0.0, use numpy.mean instead
  neg = mean(negi)


In [ ]:
print(actuals)
print("-------------------------------")
print(predicted)

0       1
1       1
2       1
3       1
4       1
       ..
1057   -1
1058   -1
1059   -1
1060   -1
1061   -1
Length: 1062, dtype: int64
-------------------------------
0      -1
1       1
2      -1
3       1
4       1
       ..
1057   -1
1058    1
1059   -1
1060    1
1061    1
Length: 1062, dtype: int64


In [ ]:
cm1=pd.crosstab(actuals, predicted, rownames=['Actuals'], colnames=['Predicted'], margins=True)
cm1

#precision score and recall scores
from sklearn.metrics import precision_score, recall_score

#Accuracy is lower than the first lexicon classifier 79% Why?
mat1 = cm1.to_numpy()
accuracy = float(mat1[0,0]+mat1[1,1])/mat1[2,2]
print('Accuracy: {0:0.3f}'.format(accuracy))

precision = precision_score(actuals, predicted)
print('Precision score: {0:0.3f}'.format(precision))
recall = recall_score(actuals, predicted)
print('Recall score: {0:0.3f}'.format(recall))

Accuracy: 0.601
Precision score: 0.584
Recall score: 0.702
